In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%config InlineBackend.figure_format='retina'

# 1d - discrete Fourier transform with numpy

- uses FFT
- assumes regular grid
- demonstrates use of `np.fft.fftfreq` to retrieve appropriate frequencies conveniently

In [ ]:
n = 1000  # number of time steps
T = 5  # Time interval
f = 2  # frequency
omega = f * 2 * np.pi  # angular frequeny

dt = T / n  # time step size
t = np.arange(0, T, dt)  # time
y = np.sin(omega * t)  # our signal y

print(f"{dt=}")

## Compute Fourier transform (`fft`)

In [ ]:
fy = np.fft.fft(y)  # Fourier transform of y
print(
    f"Size of Fourier transform data {fy.nbytes // n} bytes / data point. ({fy.dtype=})"
)

## What frequencies do we expect?

In [ ]:
fs = np.fft.fftfreq(n, d=dt)  # FrequencieS
fs[0:15]

In [ ]:
fs.shape

The spacing in frequency space is given by $df = 1 / T$:


In [ ]:
1 / T

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
ax0, ax1 = axes
ax0.plot(t, y, label="y(t)")
ax0.set_xlabel("t")
ax0.legend()

# Plot Fourier transform
ax1.plot(fs, np.abs(fy), "o-", label=r"$\mathcal{F}(y)$")
ax1.set_xlabel("frequency")
ax1.legend()
ax1.set_xlim([-5, 5]);  # Focus on peaks

In [ ]:
ind_max = np.argmax(np.abs(fy))
print(f"Maximum index is {ind_max}. Corresponding frequency is {fs[ind_max]}.")

Negative and positive frequencies are symmetric due to the realness of the data.

In [ ]:
fs[ind_max - 1 : ind_max + 2]

In [ ]:
print(f"Size of Fourier transform data {fy.nbytes // n} bytes / data point.")

## Compute Fourier transform of real data (`rfft`)

In [ ]:
rfy = np.fft.rfft(y)
print(
    f"Size of Fourier transform of real data {rfy.nbytes // n} bytes / data point. ({rfy.dtype=}, {rfy.shape=})"
)

In [ ]:
rfs = np.fft.rfftfreq(n, dt)
rfs[0:15]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
ax0, ax1 = axes
ax0.plot(t, y, label="y(t)")
ax0.set_xlabel("t")

ax1.plot(rfs, np.abs(rfy), "o-", label=r"$\mathcal{F}(y)$")
ax1.set_xlabel("frequency")

ax0.legend()
ax1.legend()
ax1.set_xlim([-5, 5]);

In [ ]:
ind_max = np.argmax(np.abs(rfy))
print(f"Maximum index is {ind_max}. Corresponding frequency is {rfs[ind_max]}.")

# xrft Approach

`xrft` has the advantage that the dimensions are also transformed and labeled during the FFT. This advantage may not seem to huge for this 1D example. But for multidimensional examples, it reduces the technical difficulties a lot. Moreover, one can take advantage of `holoviews` as well for multidimensional data when using `xrft` (s. holoviews + xrft tutorial)

In [ ]:
import xarray as xr
import xrft

In [ ]:
xa = xr.DataArray(y, coords={"t": t}, dims=("t"))

In [ ]:
xa.name = "test"  # This will not be necessary in postopus.

In [ ]:
Fxa = xrft.fft(xa, dim="t")  # numpy.fft-like behaviour

In [ ]:
# plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
ax0, ax1 = axes

xa.plot.line(ax=ax0, label="y(t)")
ax0.set_xlabel("t")

abs(Fxa).real.plot.line("o-", ax=ax1, label=r"$\mathcal{F}(y)$")
ax1.set_xlabel("frequency")

ax0.legend()
ax1.legend()
ax1.set_xlim([-5, 5])

In [ ]:
Fxar = xrft.fft(xa, dim="t", real_dim="t")  # numpy.rfft-like behaviour

In [ ]:
# plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
ax0, ax1 = axes

xa.plot.line(ax=ax0, label="y(t)")
ax0.set_xlabel("t")

abs(Fxar).plot.line("o-", ax=ax1, label=r"$\mathcal{F}(y)$")
ax1.set_xlabel("frequency")

ax0.legend()
ax1.legend()
ax1.set_xlim([-5, 5])

# Extra: xrft with corrections in amplitude and phase (numpy does not estrictly respect the theoretical phase and amplitude)

In [ ]:
Fxat = xrft.fft(xa, dim="t", true_amplitude=True, true_phase=True)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
ax0, ax1 = axes

xa.plot.line(ax=ax0, label="y(t)")
ax0.set_xlabel("t")

abs(Fxat).plot.line("o-", ax=ax1, label=r"$\mathcal{F}(y)$")
ax1.set_xlabel("frequency")

ax0.legend()
ax1.legend()
ax1.set_xlim([-5, 5]);

In [ ]:
iFxat = xrft.ifft(Fxat, dim="freq_t")
iFxat.real.plot()  # This would be shifted if one uses np.fft.

# Spectral density

In [ ]:
sd = xrft.power_spectrum(xa, dim="t", real_dim="t", scaling="density")

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(12, 4))
ax0 = axes

sd.plot.line(ax=ax0, label="Spectral density")
ax0.set_xlabel("freq_t")
ax0.set_xlim([0, 5]);